# Actividad Práctica en Clase 5


## Análisis Exploratorio de Datos


Se usó el iris dataset extraido mediante la librería de seaborn, se hará un tratamiento de datos para el posterior uso

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = sns.load_dataset("iris")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
classes = {
    "setosa": 1,
    "virginica": 2,
    "versicolor": 3
}

In [ ]:
df['species'] = df['species'].map(classes)
df.head()

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(15, 13))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True, cbar_kws={"shrink": .8})
plt.title("Matriz de Correlación")
plt.show()

### Seleccion de los datos de entrenamiento


En este caso, al ser un algoritmo de aprendizaje no supervisado, solo se suministran un conjunto de entrenamiento.
Se seleccionaron las variables correspondientes a la longitud del pétalo y a la longitud del sépalo, solo se tomaron en
cuenta estas ya que las demás tienen una colinealidad muy alta entre ellas.

In [ ]:
train = df[["sepal_length", "petal_length"]]

## Usando Kmeans

K-Means es un algoritmo de agrupamiento (clustering) no supervisado que divide un conjunto de datos en K grupos (clusters) basándose en la distancia euclidiana. Funciona de la siguiente manera:

1. Selecciona K centroides iniciales aleatorios.

2. Asigna cada punto al centroide más cercano, formando clusters.

3. Recalcula los centroides como el promedio de los puntos en cada cluster.

4. Repite hasta que los centroides ya no cambien significativamente.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(n_clusters=3)

### Entrenamiento
Se entrena el modelo y se asignan las predicciones dadas para los datos

In [ ]:
clusters = model.fit_predict(train)

In [ ]:
model.cluster_centers_

In [ ]:
classes = {
    -1: "Ruido",
    0: "Clase 1",
    1: "Clase 2",
    2: "Clase 3"
}

In [ ]:
df["cluster"] = pd.Series(clusters).map(classes)

### Visualización de los resultados del modelo de KMeans

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x='sepal_length', y='petal_length', 
                hue='cluster', palette='viridis', s=100)
plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[:, 1], 
            s=300, c='red', marker='*', label='Centroides')
plt.title('Clusters encontrados por K-means')
plt.xlabel('Longitud del sépalo')
plt.ylabel('Longitud del pétalo')
plt.legend()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x='sepal_length', y='petal_length', 
                hue='species', palette='viridis', s=100)
plt.title('Clases reales')
plt.xlabel('Longitud del sépalo')
plt.ylabel('Longitud del pétalo')
plt.legend()

Viendo las gráficas podemos observar que en su mayoría el modelo identificó los centroides y seccionó los clústeres, los
cuales son bastante parecidos a las clases reales, a excepción de algunos casos que se encuentran más cerca de otros
centroides diferentes a la clase que pertenecen.


## Usando DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

DBSCAN es un algoritmo de clustering basado en densidad, que agrupa puntos cercanos y marca como ruido aquellos en regiones de baja densidad. Funciona así:

1. Define un número mínimo de puntos (minPts) y una distancia ε (épsilon).

2. Un punto es "core" si tiene al menos minPts puntos dentro de su radio ε.

3. Los puntos alcanzables desde un "core" forman un cluster.

4. Los puntos no alcanzables se consideran ruido.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan_model = DBSCAN(eps=0.3, min_samples=10)

En este caso se usa un valor de epsilon de 0.3 y un mínimo de muestras de 10

In [ ]:
clusters = dbscan_model.fit_predict(train)

In [ ]:
df["cluster"] = pd.Series(clusters).map(classes)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x='sepal_length', y='petal_length', 
                hue='cluster', palette='plasma', s=100)
plt.title('Clusters encontrados por dbscan')
plt.xlabel('Longitud del sépalo')
plt.ylabel('Longitud del pétalo')
plt.legend()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x='sepal_length', y='petal_length', 
                hue='species', palette='plasma', s=100)
plt.title('Clases reales')
plt.xlabel('Longitud del sépalo')
plt.ylabel('Longitud del pétalo')
plt.legend()

En este caso, el modelo de DBSCAN agrupó los datos según los valores de epsilon y el mínimo de muestras. El resultado es se acerca a las clases reales, y proporciona la ventaja de que los datos que no se encuentran en el rango de proximidad del clúster se toman como ruido.

## Conclusión

El análisis demostró que para el dataset Iris, K-means es una opción simple y efectiva, especialmente cuando se conocen las características de los datos (3 especies diferenciables). DBSCAN se presenta como una alternativa más flexible para escenarios con clusters de densidad variable o cuando no se conoce el número de grupos. La elección entre ambos depende de la naturaleza de los datos y los objetivos del análisis.

La preparación de datos (selección de características no colineales) fue clave para el éxito de ambos algoritmos, mostrando la importancia del análisis exploratorio previo en tareas de clustering.